# First Steps with Pandas

In [53]:
import pandas as pd

# Importing Data

In [54]:
df = pd.read_csv('20171130_SIX_ManagementAktien.csv')

# Looking at summary of the data

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179 entries, 0 to 1178
Data columns (total 6 columns):
Unnamed: 0     1179 non-null int64
Company        1179 non-null object
Date           1179 non-null object
Price          1169 non-null object
Share Total    1179 non-null object
Type           1179 non-null object
dtypes: int64(1), object(5)
memory usage: 55.3+ KB


[Pandas data types, and Python data types](http://www.datacarpentry.org/python-ecology-lesson/03-data-types-and-format/)

# Looking at a snippet of the data

In [56]:
df.head()

,Unnamed: 0,Company,Date,Price,Share Total,Type
0,0,ABB Ltd,30.10.2017,362'229.00,14'323,Purchase
1,1,ABB Ltd,30.10.2017,304'289.00,12'032,Purchase
2,2,ABB Ltd,30.10.2017,10'060.00,500,Purchase
3,3,ABB Ltd,30.10.2017,10'060.00,500,Purchase
4,4,Banque Cantonale Vaudoise,27.10.2017,10'620.00,15,Purchase


# Deleting unnecessary data

In [57]:
del df['Unnamed: 0']

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179 entries, 0 to 1178
Data columns (total 5 columns):
Company        1179 non-null object
Date           1179 non-null object
Price          1169 non-null object
Share Total    1179 non-null object
Type           1179 non-null object
dtypes: object(5)
memory usage: 46.1+ KB


# Data Types

Sharetotal und Price sollten Zahlen sind.

In [59]:
df['Share Total'].astype(int)

ValueError: invalid literal for int() with base 10: "14'323"

## Create function and apply it

In [60]:
def comma(elem):
    elem = elem.replace("'", '')
    return elem

In [61]:
df['Share Total'] = df['Share Total'].apply(comma)

In [62]:
df['Share Total'] = df['Share Total'].astype(int)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179 entries, 0 to 1178
Data columns (total 5 columns):
Company        1179 non-null object
Date           1179 non-null object
Price          1169 non-null object
Share Total    1179 non-null int64
Type           1179 non-null object
dtypes: int64(1), object(4)
memory usage: 46.1+ KB


## Lets sort the values

In [64]:
df['Share Total'].sort_values(ascending=False).head()

1017    27205398
364      4859499
610      3496500
538      3000000
7        2190000
Name: Share Total, dtype: int64

In [65]:
df[df['Share Total']==27205398]

,Company,Date,Price,Share Total,Type
1017,KTM Industries AG,23.02.2017,133'306'450.20,27205398,Sale


## Or, if we want to see the whole sorted table 

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179 entries, 0 to 1178
Data columns (total 5 columns):
Company        1179 non-null object
Date           1179 non-null object
Price          1169 non-null object
Share Total    1179 non-null int64
Type           1179 non-null object
dtypes: int64(1), object(4)
memory usage: 46.1+ KB


## Wir wenden wieder unsere Function an, aber es gibt einen Fehler. Warum?

In [67]:
df['Price'].apply(comma)

AttributeError: 'float' object has no attribute 'replace'

In [69]:
def comma(elem):
    print(elem)
    elem = elem.replace("'", '')
    return elem

In [70]:
df['Price'].apply(comma)

362'229.00
304'289.00
10'060.00
10'060.00
10'620.00
26'700.00
123'500.00
3'816'075.00
5'760'000.00
33'518.50
4'961'000.00
4'639.38
1'069'653.00
53'627.00
1'113'112.00
1'757'500.00
1'440'000.00
392'000.00
2'350'600.00
7'875.80
558.35
63'600.00
149'500.00
256'040.60
39'600.00
840'000.00
217'800.00
11'734.00
6'220.16
411'350.00
55'099.00
148'720.00
3'697.50
54'660.00
52'712.00
236'180.75
461.00
816.00
31'206.40
68'528.25
7'811.47
827.00
37'000.00
24'750.00
71'500.00
12'269.65
5'380'300.00
9'000.00
4'747.50
91'168.00
66'000.00
32'228.00
486'020.00
388'278.55
24'750.00
9'832.50
171'538.40
24'750.00
7'500.00
29'940.90
19'170.00
1'177'000.50
19'780.00
9'920.30
7'700.00
7'700.00
6'548.00
44'875.76
64'732'500.00
9'000.00
9'758.15
24'500.00
2'549.70
125'500.00
20'801.96
57'728.00
105'590.65
184'000.00
51'163.20
55'124.00
31'248.00
118'055.23
5'696.00
6'322.50
204'000.00
4'056.50
48'360.00
36'045.00
62'950.00
2'658'977.00
126'000.00
73'065.05
23'125.00
13'440.00
947'878.00
4'917.90
112'995.00
2'5

AttributeError: 'float' object has no attribute 'replace'

In [77]:
def comma(elem):
    if elem != 'NaN':
        elem = elem.replace("'", '')
        return float(elem)
    else:
        return 0.0

In [78]:
df['Price'].apply(comma)

AttributeError: 'float' object has no attribute 'replace'

In [79]:
def comma(elem):
    try:
        elem = elem.replace("'", '')
        return float(elem)
    except:
        'AttributeError:'

In [80]:
df['Price'].apply(comma)

0        362229.00
1        304289.00
2         10060.00
3         10060.00
4         10620.00
5         26700.00
6        123500.00
7       3816075.00
8       5760000.00
9         33518.50
10      4961000.00
11         4639.38
12      1069653.00
13        53627.00
14      1113112.00
15      1757500.00
16      1440000.00
17       392000.00
18      2350600.00
19         7875.80
20          558.35
21        63600.00
22       149500.00
23       256040.60
24        39600.00
25       840000.00
26       217800.00
27        11734.00
28         6220.16
29       411350.00
           ...    
1149        286.30
1150    8316000.00
1151      79632.00
1152      33768.00
1153      33012.00
1154      30012.00
1155     934571.54
1156     246654.00
1157      36544.17
1158        641.12
1159       5139.75
1160       1026.00
1161      11053.48
1162      12500.00
1163      12261.40
1164      19045.83
1165        285.33
1166      15270.00
1167      10100.00
1168      20535.74
1169        351.63
1170      27

## Another way of dealing with NaN Values -> Drop them

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179 entries, 0 to 1178
Data columns (total 5 columns):
Company        1179 non-null object
Date           1179 non-null object
Price          1169 non-null object
Share Total    1179 non-null int64
Type           1179 non-null object
dtypes: int64(1), object(4)
memory usage: 46.1+ KB


In [83]:
df = df.dropna()

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1169 entries, 0 to 1178
Data columns (total 5 columns):
Company        1169 non-null object
Date           1169 non-null object
Price          1169 non-null float64
Share Total    1169 non-null int64
Type           1169 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 94.8+ KB


[Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html)

In [88]:
df['Price'] = df['Price'].apply(comma)

In [92]:
df.sort_values('Price',ascending=False).head()

,Company,Date,Price,Share Total,Type
1017,KTM Industries AG,23.02.2017,1.333065e+08,27205398,Sale
885,Bank Coop AG,22.03.2017,7.799227e+07,1751848,Purchase
364,Idorsia Ltd,19.06.2017,6.976583e+07,4859499,Purchase
68,Feintool International Holding AG,29.09.2017,6.473250e+07,616500,Sale
312,Idorsia Ltd,07.07.2017,3.062190e+07,1531095,Purchase


Price in Millions

In [94]:
df['Price_m'] = df['Price'] / 1000000

## Who sold the most?

In [98]:
df.groupby('Company')['Price_m'].sum().sort_values()

Company
Basler Kantonalbank                        0.000966
Züblin Immobilien Holding AG               0.002678
Emmi AG                                    0.003430
Groupe Minoteries SA                       0.005546
Warteck Invest AG                          0.005853
Alpiq Holding AG                           0.008200
Cicor Technologies Ltd.                    0.010950
Kuros Biosciences AG                       0.012550
Orell Füssli Holding AG                    0.012667
Perrot Duval Holding SA                    0.017160
Airesis SA                                 0.018528
Thurgauer Kantonalbank                     0.019780
LumX Group Limited                         0.025500
Mikron Holding AG                          0.032300
SFS Group AG                               0.032790
BKW AG                                     0.045360
EFG International                          0.045547
Myriad Group AG                            0.047500
Kardex AG                                  0.048679
Star